WARNING: this example is not yet fully functional

In [1]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = np.empty(3, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Queue(model, 'Queue2', SchedStrategy.FCFS)

jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 1, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 2, node[0], 0)

# renewal
map21 = APH([1,0],[[-2,2],[0,-0.5]])
map22 = MAP([-1],[1]) 
# non-renewal
map31 = MAP([[-20,0],[0,-1]],[[0,20],[0.8,0.2]])
map32 = MAP([[-20,0],[0,-1]],[[0,20],[0.8,0.2]])

node[0].setService(jobclass[0], HyperExp.fitMeanAndSCV(1.0,25.0))
node[1].setService(jobclass[0], map21)
node[2].setService(jobclass[0], map31)

node[0].setService(jobclass[1], HyperExp.fitMeanAndSCV(1.0,25.0))
node[1].setService(jobclass[1], map22)
node[2].setService(jobclass[1], map32)

model.addLink(node[0], node[0])
model.addLink(node[0], node[1])
model.addLink(node[0], node[2])
model.addLink(node[1], node[0])
model.addLink(node[2], node[0])

node[0].setRouting(jobclass[0],RoutingStrategy.RROBIN) # if changed to RAND the EJML exception goes
node[1].setProbRouting(jobclass[0], node[0], 1.0)
node[2].setProbRouting(jobclass[0], node[0], 1.0)

node[0].setRouting(jobclass[1],RoutingStrategy.RROBIN) # if changed to RAND the EJML exception goes
node[1].setProbRouting(jobclass[1], node[0], 1.0)
node[2].setProbRouting(jobclass[1], node[0], 1.0)

In [3]:
solver = np.array([], dtype=object)
#solver = np.append(solver, SolverCTMC(model,'keep',False))

In [4]:
#TODO: bug in SolverJMT in handling this case: the state of the source is [1 0 2 0] rather than [1 0 2 0 1 1] and nvars in the third row is all zeros but it should be [1     1     0     0     0]
#solver = np.append(solver, SolverJMT(model,'seed',23000,'samples',100000,'keep',True))

In [5]:
#TODO: same bug as above
#solver = np.append(solver, SolverSSA(model,'seed',23000,'verbose',True,'samples',50000))

In [6]:
AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].getName()}')
    AvgTable[s] = solver[s].getAvgTable()
    print(AvgTable[s])